In [ ]:
# 1. 필요한 데이터 가져오기
import pandas as pd

data = pd.read_excel("/content/drive/MyDrive/Project/data.xlsx", engine="openpyxl")

data.head()

In [24]:
# 2. 데이터 분할 (train 훈련 / test 검증)
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)
train_data.shape, test_data.shape

((12580, 2), (3146, 2))

In [ ]:
!pip install transformers

In [26]:
# 3. 전처리 : sentence의 한국어를 숫자벡터로 변환하는 토큰화 작업
# Hugging face의 BERT모델 활용

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

# 토크나이저 설정
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

# 데이터셋 정의
class TextDataset(Dataset):
  def __init__(self, sentences, labels, tokenizer, max_len):
    self.sentences = sentences
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self, item):
    sentence = str(self.sentences[item])
    label = self.labels[item]

    encoding = self.tokenizer.encode_plus(
        sentence,
        add_special_tokens = True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask = True,
        return_tensors = 'pt'
      )

    return {
        'sentence' : sentence,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten(),
        'labels' : torch.tensor(label, dtype = torch.long)
      }

In [27]:
# 데이터 셋 생성
max_len = 128    # 최대 시퀀스 길이 설정
train_dataset = TextDataset(
    train_data['sentence'].to_numpy(),
    train_data['label_idx'].to_numpy(),
    tokenizer,
    max_len
)

test_dataset = TextDataset(
    train_data['sentence'].to_numpy(),
    train_data['label_idx'].to_numpy(),
    tokenizer,
    max_len
)

In [33]:
# 데이터 로더 생성
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# BERT 모델 로드 및 학습 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels = len(data['label_idx'].unique()))
model.to(device)

In [35]:
from torch.optim import AdamW
# 옵티마이저 설정
optimizer = torch.optim.AdamW(model.parameters(),lr=5e-5)
# 손실함수 설정
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

epochs = 20
for epoch in range(epochs):
  # 훈련
  model.train()
  for batch in tqdm(train_dataloader, desc=f'Train epoch : {epoch + 1}'):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # 포워드 태우기
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # 백워드 태우기
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch {epoch +1}/{epochs} completed')

  # 평가
  model.eval()
  total_loss = 0
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for batch in tqdm(test_dataloader, desc=f'Test epoch : {epoch +1}'):
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs.loss
      total_loss += loss.item()

      # 예측
      _, preds = torch.max(outputs.logits, dim=1)
      all_preds.extend(preds.tolist())
      all_labels.extend(labels.tolist())

    avg_loss = total_loss / len(test_dataloader)
    avg_accuracy = accuracy_score(all_labels, all_preds)
    avg_precision = precision_score(all_labels, all_preds, average = 'weighted')
    avg_recall = recall_score(all_labels, all_preds, average = 'weighted')
    avg_f1 = f1_score(all_labels, all_preds, average = 'weighted')

    print(f'Test Loss : {avg_loss}')
    print(f'Test accuracy : {avg_accuracy}')
    print(f'Test f1 score : {avg_f1}')
    print(f'Test precision : {avg_precision}')
    print(f'Test recall : {avg_recall}')

Train epoch : 1:   0%|          | 0/197 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train epoch : 1: 100%|██████████| 197/197 [03:48<00:00,  1.16s/it]


Epoch 1/20 completed


Test epoch : 1: 100%|██████████| 197/197 [01:23<00:00,  2.36it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Loss : 1.8881846273005916
Test accuracy : 0.747933227344992
Test f1 score : 0.6752091605135122
Test precision : 0.6333528737084045
Test recall : 0.747933227344992


Train epoch : 2:   0%|          | 0/197 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train epoch : 2: 100%|██████████| 197/197 [03:46<00:00,  1.15s/it]


...

Train epoch : 15:   0%|          | 0/197 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train epoch : 15: 100%|██████████| 197/197 [03:46<00:00,  1.15s/it]


Epoch 15/20 completed


Test epoch : 15: 100%|██████████| 197/197 [01:23<00:00,  2.37it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Loss : 0.24217756761966017
Test accuracy : 0.978060413354531
Test f1 score : 0.9736171302902843
Test precision : 0.9710548214036997
Test recall : 0.978060413354531


Train epoch : 16:   0%|          | 0/197 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Train epoch : 16:   2%|▏         | 4/197 [00:04<03:48,  1.18s/it]